In [1]:
import pandas as pd
import numpy as np
import locale
import os
from datetime import datetime
import warnings
locale.setlocale(locale.LC_ALL, 'pt_BR.UTF-8')
warnings.filterwarnings('ignore')

# Configurações do Pandas
pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.precision', 2)

## Importações do Banco de dados

In [2]:
%%sql
SELECT
    ano as ANO,
    mes AS MES,
    data AS DATA,
    numeroNotaFiscal AS NUMERO_NF,
    destino AS DESTINO,
    tipo AS TIPO,
    procedencia AS PROCEDENCIA,
    estado AS ESTADO,
    produto AS PRODUTO,
    unidade AS UNIDADE,
    unidade_peso AS UNIDADE_PESO,
    quantidade AS QUANTIDADE,
    volume AS VOLUME
FROM
    (
        SELECT
            EXTRACT(YEAR FROM data) AS ano,
            EXTRACT(MONTH FROM data) AS mes,
            data,
            numeroNotaFiscal,
            destino,
            procedencia,
            produto,
            unidade,
            unidade_peso,
            quantidade,
            volume
        FROM divtec_notasfiscais

        UNION ALL

        SELECT
            EXTRACT(YEAR FROM data) AS ano,
            EXTRACT(MONTH FROM data) AS mes,
            data,
            numeroNotaFiscal,
            destino,
            procedencia,
            produto,
            unidade,
            unidade_peso,
            quantidade,
            volume
        FROM notas_temporarias
    ) AS uniao
LEFT JOIN
    cad_cidades ON cad_cidades.cidade = uniao.procedencia
LEFT JOIN
    cad_usuarios ON uniao.destino = cad_usuarios.matricula_nome
ORDER BY ano, mes, data, destino, numeroNotaFiscal, produto;


,ANO,MES,DATA,NUMERO_NF,DESTINO,TIPO,PROCEDENCIA,ESTADO,PRODUTO,UNIDADE,UNIDADE_PESO,QUANTIDADE,VOLUME
0,2024,1,2024-01-02,122832,0100000-PRODUTORES,PRODUTOR,0143188-CRUZEIRO DO SUL (RS),RIO GRANDE DO SUL,06701-MILHO VERDE (3UN),KG,1.00,250.00,250.00
1,2024,1,2024-01-02,134608,0100000-PRODUTORES,PRODUTOR,0143128-SAO SEBASTIAO DO CAI (RS),RIO GRANDE DO SUL,04702-BROCOLIS,DZ,6.00,10.00,60.00
2,2024,1,2024-01-02,134608,0100000-PRODUTORES,PRODUTOR,0143047-FELIZ (RS),RIO GRANDE DO SUL,07002-PEPINO SALADA,CX,20.00,18.00,360.00
3,2024,1,2024-01-02,134608,0100000-PRODUTORES,PRODUTOR,0143128-SAO SEBASTIAO DO CAI (RS),RIO GRANDE DO SUL,07003-PEPINO JAPONES,CX,20.00,5.00,100.00
4,2024,1,2024-01-02,134608,0100000-PRODUTORES,PRODUTOR,0143128-SAO SEBASTIAO DO CAI (RS),RIO GRANDE DO SUL,07201-PIMENTAO,CX,10.00,8.00,80.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
321493,2024,12,2024-12-30,12920,5901948-TDR COMERCIO DE HORTIFRUTIGRANJEIROS LTDA,ATACADISTA,0143386-PARECI NOVO (RS),RIO GRANDE DO SUL,01401-LARANJA BAIA (UMBIGO),CX,20.00,10.00,200.00
321494,2024,12,2024-12-30,12920,5901948-TDR COMERCIO DE HORTIFRUTIGRANJEIROS LTDA,ATACADISTA,0143386-PARECI NOVO (RS),RIO GRANDE DO SUL,01403-LARANJA LIMA (CEU),CX,20.00,10.00,200.00
321495,2024,12,2024-12-30,12920,5901948-TDR COMERCIO DE HORTIFRUTIGRANJEIROS LTDA,ATACADISTA,0143386-PARECI NOVO (RS),RIO GRANDE DO SUL,01406-LARANJA SUCO,CX,20.00,80.00,"1,600.00"
321496,2024,12,2024-12-30,12920,5901948-TDR COMERCIO DE HORTIFRUTIGRANJEIROS LTDA,ATACADISTA,0143386-PARECI NOVO (RS),RIO GRANDE DO SUL,01603-LIMAO TAITI,CX,20.00,10.00,200.00


Ajustes de tipos de dados nas colunas

In [3]:
df_notas_raw[['NUMERO_NF','TIPO','PROCEDENCIA','PRODUTO','ESTADO','UNIDADE']] = df_notas_raw[['NUMERO_NF','TIPO','PROCEDENCIA','PRODUTO','ESTADO','UNIDADE']].astype(str)
df_notas_raw['DATA'] = df_notas_raw['DATA'].astype('datetime64[ns]')
# df_notas_raw.to_parquet('notas_fiscais.parquet')

## Verificação dos dados de Cotação

In [4]:
%%sql
SELECT produto AS PRODUTO, valor as PESO, tipo AS UNIDADE FROM divtec_pesos_produtos_pivot;

,PRODUTO,PESO,UNIDADE
0,00101-ABACATE,1.0,KG
1,00201-ABACAXI CAIENA (HAWAI),1.0,KG
2,00202-ABACAXI PEROLA,1.0,KG
3,00203-ABACAXI TERRA DE AREIA,1.0,KG
4,00301-AMEIXA NACIONAL,1.0,KG
...,...,...,...
2718,FRAMBOESA,NaN,SCL
2719,FRAMBOESA,NaN,SACO
2720,FRAMBOESA,NaN,BDJ
2721,FRAMBOESA,NaN,UND


In [5]:
df_cotacoes = pd.read_csv('../../Bases de dados/Cotacao 2024  Jessica.csv', sep=';', parse_dates=['DATA'], dayfirst=True)
df_cotacoes['UNIDADE'] = df_cotacoes['UNIDADE'].astype(str)
df_cotacoes = pd.merge(df_cotacoes, df_produtos_pesos, on=['PRODUTO', 'UNIDADE'], how='left')
df_cotacoes['PESO / KG'] = df_cotacoes['MAIS FREQUENTE'] / df_cotacoes['PESO']
df_cotacoes = df_cotacoes[['DATA','PRODUTO','UNIDADE','PESO / KG']]
df_cotacoes['UNIDADE'] = df_cotacoes['UNIDADE'].apply(lambda x: x.upper())
df_cotacoes

,DATA,PRODUTO,UNIDADE,PESO / KG
0,2024-03-20,00000-FOLHAGENS VARIADAS,UND,10.0
1,2024-04-22,00000-FOLHAGENS VARIADAS,UND,10.0
2,2024-05-30,00000-FOLHAGENS VARIADAS,UND,10.0
3,2024-05-31,00000-FOLHAGENS VARIADAS,UND,10.0
4,2024-06-03,00000-FOLHAGENS VARIADAS,UND,10.0
...,...,...,...,...
54083,2024-12-30,TAMARINDO,KG,NaN
54084,2024-12-26,TOMATE RASTEIRO,KG,NaN
54085,2024-12-27,TOMATE RASTEIRO,KG,NaN
54086,2024-12-29,TOMATE RASTEIRO,KG,NaN


In [6]:
# Fazendo o merge dos DataFrames
dados_combinados = pd.merge(df_notas_raw, df_cotacoes, on=['DATA', 'PRODUTO','UNIDADE'], how='left', indicator=True)

# Filtrando os dados que possuem nota fiscal mas não têm cotação
notas_sem_cotacao = dados_combinados[dados_combinados['_merge'] == 'left_only']

# Removendo a coluna '_merge'
notas_sem_cotacao = notas_sem_cotacao.drop(columns=['_merge'])

# Mostrando as combinações de data e produto que possuem nota fiscal mas não têm cotação
produtos_sem_cotacao = notas_sem_cotacao[['DATA', 'PRODUTO']].drop_duplicates()

produtos_sem_cotacao


,DATA,PRODUTO
1,2024-01-02,07002-PEPINO SALADA
2,2024-01-02,07003-PEPINO JAPONES
3,2024-01-02,07201-PIMENTAO
6,2024-01-02,12301-FLORES COMESTIVEIS
8,2024-01-02,00503-BANANA PRATA / BRANCA
...,...,...
321580,2024-12-30,52006-KALANCHOE MINI15UN
321582,2024-12-30,08001-CEBOLA NACIONAL
321583,2024-12-30,51701-CRISANTEMO
321604,2025-04-25,05601-REPOLHO VERDE


In [ ]:
# produtos_sem_cotacao.to_excel('Faltando Cotações.xlsx', index=False)

## Notas Fiscais

In [ ]:
df_nfs = df_notas_raw.copy(deep=True)

### Notas Fiscais Dados Extremos

In [ ]:
df_nfs = df_nfs.loc[df_nfs['volume'] >= 40000]
df_nfs.to_excel('Valores Errados.xlsx', index=False)

## Valor Comercializado

In [7]:
df = df_notas_raw.copy(deep=True)
df = df[['DATA','PRODUTO','VOLUME']]
# df = df.rename(columns={'data': 'DATA', 'produto': 'PRODUTO', 'volume':'VOLUME'})
df = df.groupby(['DATA','PRODUTO'])['VOLUME'].sum().reset_index()

valor_comercializado = pd.merge(left=df, right=df_cotacoes, how='left', on=['DATA','PRODUTO'])
valor_comercializado['TOTAL'] = valor_comercializado['VOLUME'] * valor_comercializado['PESO / KG']
valor_comercializado = valor_comercializado.loc[~(valor_comercializado['TOTAL'].isnull())]
valor_comercializado

,DATA,PRODUTO,VOLUME,UNIDADE,PESO / KG,TOTAL
0,2024-01-02,00101-ABACATE,27420.00,KG,5.000000,137100.000000
1,2024-01-02,00202-ABACAXI PEROLA,147710.00,UND,6.153846,908984.615385
2,2024-01-02,00301-AMEIXA NACIONAL,29385.00,KG,8.000000,235080.000000
3,2024-01-02,00350-AMEIXA IMPORTADA,9.00,KG,12.220000,109.980000
4,2024-01-02,00401-AMORA PRETA,70.00,KG,16.670000,1166.900000
...,...,...,...,...,...,...
28177,2024-12-30,52006-KALANCHOE MINI15UN,47.00,CX,30.000000,1410.000000
28178,2024-12-30,52601-SUCULENTAS,54.00,UND,35.000000,1890.000000
28179,2024-12-30,54601-ROSA VERMELHA 20UN,122.60,MOL,132.508834,16245.583039
28180,2025-04-25,05601-REPOLHO VERDE,308.52,KG,1.600000,493.632000


In [15]:
# Substituir np.inf, -np.inf por NaN e NaN por 0
valor_comercializado['TOTAL'] = valor_comercializado['TOTAL'].replace([np.inf, -np.inf], np.nan)
valor_comercializado['TOTAL'] = valor_comercializado['TOTAL'].fillna(0)

# Garantir que a coluna é numérica
valor_comercializado['TOTAL'] = pd.to_numeric(valor_comercializado['TOTAL'], errors='coerce')

# Somar a coluna TOTAL
total_sum = valor_comercializado['TOTAL'].sum()

print(f"Soma total da coluna TOTAL: {total_sum}")


Soma total da coluna TOTAL: 2281388538.155628


In [16]:
df2 = df_notas_raw.copy(deep=True)
df2 = pd.merge(df2, df_cotacoes, how='left', on=['DATA','PRODUTO'])
# df2 = df2['PESO / KG'].fillna(0)
df2['VALOR'] = df2['VOLUME'] * df2['PESO / KG'].fillna(0)
df2['VALOR'] = df2['VALOR'].astype('float64')
df2

,ANO,MES,DATA,NUMERO_NF,DESTINO,TIPO,PROCEDENCIA,ESTADO,PRODUTO,UNIDADE_x,UNIDADE_PESO,QUANTIDADE,VOLUME,UNIDADE_y,PESO / KG,VALOR
0,2024,1,2024-01-02,134608,0100000-PRODUTORES,PRODUTOR,0143128-SAO SEBASTIAO DO CAI (RS),RIO GRANDE DO SUL,04702-BROCOLIS,DZ,6.00,10.00,60.00,DZ,8.33,500.00
1,2024,1,2024-01-02,134608,0100000-PRODUTORES,PRODUTOR,0143047-FELIZ (RS),RIO GRANDE DO SUL,07002-PEPINO SALADA,CX,20.00,18.00,360.00,KG,3.33,"1,198.80"
2,2024,1,2024-01-02,134608,0100000-PRODUTORES,PRODUTOR,0143128-SAO SEBASTIAO DO CAI (RS),RIO GRANDE DO SUL,07003-PEPINO JAPONES,CX,20.00,5.00,100.00,KG,4.44,444.00
3,2024,1,2024-01-02,134608,0100000-PRODUTORES,PRODUTOR,0143128-SAO SEBASTIAO DO CAI (RS),RIO GRANDE DO SUL,07201-PIMENTAO,CX,10.00,8.00,80.00,KG,5.50,440.00
4,2024,1,2024-01-02,21096,0100003-TAKAYOSHI SATO,PRODUTOR,0143057-GRAVATAI (RS),RIO GRANDE DO SUL,04801-CHEIRO VERDE / TEMPERO,DZ,0.50,20.00,10.00,DZ,30.00,300.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
321640,2024,12,2024-12-30,12920,5901948-TDR COMERCIO DE HORTIFRUTIGRANJEIROS LTDA,ATACADISTA,0143386-PARECI NOVO (RS),RIO GRANDE DO SUL,01406-LARANJA SUCO,CX,20.00,80.00,"1,600.00",KG,4.16,"6,656.00"
321641,2024,12,2024-12-30,12920,5901948-TDR COMERCIO DE HORTIFRUTIGRANJEIROS LTDA,ATACADISTA,0143386-PARECI NOVO (RS),RIO GRANDE DO SUL,01603-LIMAO TAITI,CX,20.00,10.00,200.00,KG,3.05,610.00
321642,2024,12,2024-12-30,12920,5901948-TDR COMERCIO DE HORTIFRUTIGRANJEIROS LTDA,ATACADISTA,0129183-JUAZEIRO (BA),BAHIA,01901-MANGA,CX,20.00,20.00,400.00,KG,3.61,"1,444.00"
321643,2025,4,2025-04-25,589645,0113827-VALDOMIRO STRATSBURG TIETBOHL,PRODUTOR,0143420-TRES FORQUILHAS (RS),RIO GRANDE DO SUL,05601-REPOLHO VERDE,CX,15.43,20.00,308.52,KG,1.60,493.63


In [17]:
print(f"{df2['VOLUME'].sum()}")
print(f"{df2['VALOR'].sum()}")

453144958.4229999
inf


## Atacadistas

In [4]:
df = df_notas_raw.copy(deep=True)
lista = [
	'5501063-VALMOR ERTHAL FRUTARIA','5500939-COMERCIAL HORTIFR. FATIM',
	'5500991-ALCIDES SOTER DE OLIVERA (CATARINA)','5500971-CHS COMERCIO DE FRUTAS E',
	'5501007-COMERCIAL ATACADISTA SIMON','5500971-CHS COMERCIO DE FRUTAS E',
	'5500940-UNISALVO COM. DE HORT. E', '5500020-BRAULIO COMERCIAL DE FR.E LEG. LTDA',
	'5500870-JACOBY COM. DE LEG. E HO','5500820-PORTAL DAS FRUTAS LTDA',
	'5500022-SILVESTRIN IMPORTADORA DE FRUTAS LTDA','5501059-ERS COM. DE ALIMENTOS LT',
	'5500091-M B MOYSES & CIA LTDA','5500952-WINTER COM. DE ALIM. E T'

]
df = df.loc[df['DESTINO'].isin(lista)]
df

,ANO,MES,DATA,NUMERO_NF,DESTINO,TIPO,PROCEDENCIA,ESTADO,PRODUTO,UNIDADE,UNIDADE_PESO,QUANTIDADE,VOLUME
1125,2024,1,2024-01-02,769014,5500022-SILVESTRIN IMPORTADORA DE FRUTAS LTDA,ATACADISTA,0143045-FARROUPILHA (RS),RIO GRANDE DO SUL,00301-AMEIXA NACIONAL,KG,1.00,27.00,27.00
1126,2024,1,2024-01-02,769014,5500022-SILVESTRIN IMPORTADORA DE FRUTAS LTDA,ATACADISTA,0800000-(ARGENTINA),ARGENTINA,00350-AMEIXA IMPORTADA,KG,1.00,9.00,9.00
1127,2024,1,2024-01-02,769014,5500022-SILVESTRIN IMPORTADORA DE FRUTAS LTDA,ATACADISTA,0143045-FARROUPILHA (RS),RIO GRANDE DO SUL,01705-MACA GALA CX PAP,KG,1.00,18.00,18.00
1128,2024,1,2024-01-02,769014,5500022-SILVESTRIN IMPORTADORA DE FRUTAS LTDA,ATACADISTA,0800000-(ARGENTINA),ARGENTINA,01755-MACA RED DELICIOUS,KG,1.00,48.00,48.00
1129,2024,1,2024-01-02,769014,5500022-SILVESTRIN IMPORTADORA DE FRUTAS LTDA,ATACADISTA,0143045-FARROUPILHA (RS),RIO GRANDE DO SUL,02401-NECTARINA,KG,1.00,351.00,351.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
320687,2024,12,2024-12-29,585608,5501063-VALMOR ERTHAL FRUTARIA,ATACADISTA,0143029-CAXIAS DO SUL (RS),RIO GRANDE DO SUL,11001-ALHO PORO,DZ,3.00,40.00,120.00
321263,2024,12,2024-12-30,843244,5500022-SILVESTRIN IMPORTADORA DE FRUTAS LTDA,ATACADISTA,0143045-FARROUPILHA (RS),RIO GRANDE DO SUL,11001-ALHO PORO,KG,1.00,13.00,13.00
321413,2024,12,2024-12-30,25,5500971-CHS COMERCIO DE FRUTAS E,ATACADISTA,0143159-VIAMAO (RS),RIO GRANDE DO SUL,07404-TOMATE CAQUI LONGA VIDA,CX,20.00,400.00,"8,000.00"
321472,2024,12,2024-12-30,63,5501063-VALMOR ERTHAL FRUTARIA,ATACADISTA,0135599-TAQUARIVAI (SP),SAO PAULO,07201-PIMENTAO,CX,10.00,90.00,900.00


### Volumes Atacadistas

In [7]:
volumes_atacadistas = df.copy(deep=True)
volumes_atacadistas = volumes_atacadistas.groupby(['DESTINO','MES'])['VOLUME'].sum().reset_index()
volumes_atacadistas = volumes_atacadistas.pivot_table(index='DESTINO', columns='MES', values='VOLUME')
volumes_atacadistas = volumes_atacadistas.fillna(0)
volumes_atacadistas['TOTAL'] = volumes_atacadistas.sum(axis=1)
volumes_atacadistas = volumes_atacadistas.sort_values(by='TOTAL', ascending=False)
# volumes_atacadistas.to_excel('Volume de Tomates comercializado Atacadistas.xlsx')
volumes_atacadistas

MES,1,2,3,4,5,6,7,8,9,10,11,12,TOTAL
DESTINO,,,,,,,,,,,,,
5501007-COMERCIAL ATACADISTA SIMON,"1,007,837.10","768,810.00","545,973.00","825,737.00","41,860.00","350,471.50","794,830.64","643,546.00","931,652.00","824,312.00","780,344.00","669,675.00","8,185,048.24"
5501059-ERS COM. DE ALIMENTOS LT,"335,770.00","369,970.00","369,556.00","449,151.00",0.00,"97,280.00","812,989.00","572,010.00","469,428.00","595,630.00","734,733.00","364,394.00","5,170,911.00"
5500820-PORTAL DAS FRUTAS LTDA,"343,680.00","382,177.70","322,906.00","442,919.20","28,897.30","266,126.60","304,975.70","332,391.30","373,540.50","435,994.56","503,179.70","591,463.30","4,328,251.86"
5500940-UNISALVO COM. DE HORT. E,"514,447.00","433,877.00","371,626.00","294,584.00",0.00,"122,686.60","328,568.00","172,764.00","183,521.00","430,437.00","456,889.00","273,723.00","3,583,122.60"
5500971-CHS COMERCIO DE FRUTAS E,"190,900.00","121,950.00","98,130.00","210,490.00",0.00,"85,600.00","349,906.00","394,730.00","475,055.00","424,980.00","412,830.00","253,820.00","3,018,391.00"
5500022-SILVESTRIN IMPORTADORA DE FRUTAS LTDA,"339,606.00","375,536.90","332,565.30","305,401.70","32,769.42","83,968.20","180,128.00","255,847.85","259,463.60","297,900.50","196,921.12","170,386.80","2,830,495.39"
5500091-M B MOYSES & CIA LTDA,"139,398.00","110,263.00","81,510.00","161,170.00","68,635.00","92,976.60","470,360.00","372,794.00","368,822.00","298,500.00","220,130.00","176,647.00","2,561,205.60"
5500991-ALCIDES SOTER DE OLIVERA (CATARINA),"76,120.00","22,660.00","30,800.00","90,710.00","103,980.00","189,460.00","299,387.00","227,925.00","312,185.00","258,350.00","219,040.00","96,180.00","1,926,797.00"
5501063-VALMOR ERTHAL FRUTARIA,"164,435.00","101,350.00","156,586.00","117,995.00","94,910.00","74,957.00","116,450.00","227,060.00","246,840.00","202,230.00","228,178.00","76,685.00","1,807,676.00"


### Notas Fiscais

In [14]:
notas_fiscais = df.copy(deep=True)
notas_fiscais = notas_fiscais[['DATA','MES','DESTINO','NUMERO_NF','VOLUME']].drop_duplicates()
notas_fiscais = notas_fiscais.sort_values(['DESTINO','DATA','NUMERO_NF','VOLUME'])
notas_fiscais = notas_fiscais.groupby(['DESTINO','MES'])[['NUMERO_NF','VOLUME']].agg({
	'NUMERO_NF': 'count',
	'VOLUME': 'sum'
})

notas_fiscais = notas_fiscais.pivot_table(index='DESTINO', columns='MES', values=['NUMERO_NF', 'VOLUME'])
notas_fiscais.columns = [f'{col[1]}_{col[0]}' for col in notas_fiscais.columns]
notas_fiscais = notas_fiscais.reindex(sorted(notas_fiscais.columns, key=lambda x: (int(x.split('_')[0]), x.split('_')[1])), axis=1)
notas_fiscais = notas_fiscais.fillna(0)

notas_fiscais['SOMA_NUMERO_NF'] = notas_fiscais.filter(like='NUMERO_NF').sum(axis=1)
notas_fiscais['SOMA_VOLUME'] = notas_fiscais.filter(like='VOLUME').sum(axis=1)

notas_fiscais.to_excel('Listagem de Notas Fiscais Atacadistas Selecionados.xlsx')
# notas_fiscais
